# EXIF tag writer 

In [ ]:
import os
from pathlib import Path
from joblib import delayed, Parallel
import glob
import tqdm
import numpy as np

In [ ]:
%config Completer.use_jedi = False

In [ ]:
DATA_DIR = Path(r'D:\pix4d\Cape_Blossom_2021-06-25\01_rawdata\tif')
outdir = {'right': DATA_DIR / Path('33576_Right'),
          'left':DATA_DIR / Path('33577_Left'),
          'nir':DATA_DIR / Path('33552_NIR')}
exifpath = Path(r'exiftool\exiftool.exe')
tag = {'right':'MACS_RGB_Right_33576',
       'left':'MACS_RGB_Left_33577',
       'nir':'MACS_NIR_33552'}

#### no parallelization necessary 

In [ ]:
def write_exif(outdir, tag, exifpath):
    s = f'{exifpath} -overwrite_original -Model="{tag}" {outdir}'
    print(s)
    os.system(s)

In [ ]:
for sensor in ['right', 'left', 'nir']:
    %time write_exif(outdir[sensor], tag[sensor], exifpath)

### Rasterio read image stats
* quick with parallel reading

In [ ]:
import rasterio

In [ ]:
flist = list(outdir['nir'].glob('*.tif'))

In [ ]:
def get_stats(filepath):
    with rasterio.open(filepath) as src:
        d = src.read()
        return d.min(), d.mean(), d.max()

In [ ]:
%time stats = Parallel(n_jobs=30)(delayed(get_stats)(f) for f in flist)

## Edit mipps 

In [ ]:
from xml.dom.minidom import parse, parseString

In [ ]:
import xml.dom.minidom as minidom

In [ ]:
minidom.Node.toxml()

In [ ]:
mippsfile = Path('mipps_scripts') / '33576_all_taps_linearstretch.mipps'

In [ ]:
parseString("
<QVariantMap>
    <QVariantMap name="Filter">
        <bool name="enabled" value="true"/>
        <double name="max" value="30"/>
        <double name="min" value="0"/>
        <QString name="name" value="Linear-Stretch"/>
    </QVariantMap>
</QVariantMap>
")

In [ ]:
dom = parse(open(mippsfile)).documentElement

In [ ]:
filters = dom.childNodes[1].childNodes[1].childNodes

In [ ]:
f1 = filters[1]

#### Exiftool parallelization tests 

In [ ]:
def write_exif_parallel(outdir, tag, exifpath):
    flist = outdir.glob('*.tif')
    for f in flist:
        s = f'{exifpath} -overwrite_original -Model="{tag}" {outdir}'
        print(s)
        os.system(s)

In [ ]:
def write_exiffile(outfile, tag, exifpath):
    s = f'{exifpath} -overwrite_original -Model="{tag}" {outfile}'
    os.system(s)

def write_exif_parallel2(outdir, tag, exifpath):
    flist = outdir.glob('*.tif')
    Parallel()(delayed(write_exiffile)(f, tag, exifpath) for f in tqdm.tqdm_notebook(flist))

In [ ]:
%time write_exif_parallel(outdir, tag, exifpath)

In [ ]:
%time write_exif_parallel2(outdir, tag, exifpath)